##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar avaliações

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/evaluator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/evaluator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/evaluator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/evaluator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

A avaliação é uma parte crítica dos modelos de medição e benchmarking.

Este guia demonstra como migrar tarefas do avaliador (evaluator) do TensorFlow 1 para o TensorFlow 2. No Tensorflow 1, essa funcionalidade é implementada por `tf.estimator.train_and_evaluate`, quando a API está sendo executada de forma distribuída. No Tensorflow 2, você pode usar o `tf.keras.utils.SidecarEvaluator` integrado ou um loop de avaliação personalizado na tarefa do avaliador.

Existem alternativas simples de avaliação serial tanto no TensorFlow 1 (`tf.estimator.Estimator.evaluate`) como no TensorFlow 2 (`Model.fit(..., validation_data=(...))` ou `Model.evaluate`). A tarefa do avaliador é preferível quando você deseja que seus workers não fiquem alternando entre treinamento e avaliação, e a avaliação integrada no `Model.fit` é preferível quando você deseja que sua avaliação seja feita de forma distribuída.


## Configuração

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import numpy as np
import tempfile
import time
import os

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## TensorFlow 1: avaliação usando tf.estimator.train_and_evaluate

No TensorFlow 1, você pode configurar um `tf.estimator` para avaliar o estimador usando `tf.estimator.train_and_evaluate`.

Neste exemplo, comece definindo o `tf.estimator.Estimator` e definindo as especificações de treinamento e avaliação:

In [ ]:
feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.2
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

Em seguida, treine e avalie o modelo. A avaliação é feita de forma síncrona entre treinamentos porque é limitada como uma execução local neste notebook e alterna entre treinamento e avaliação. No entanto, se o estimador for usado de forma distribuída, o avaliador será executado como uma tarefa de avaliador dedicada. Para mais informações, consulte o [guia de migração em treinamento distribuído](https://www.tensorflow.org/guide/migrate/multi_worker_cpu_gpu_training).

In [ ]:
tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

## TensorFlow 2: avaliando um modelo Keras

No TensorFlow 2, se você usar a API Keras `Model.fit` para treinamento, poderá avaliar o modelo com `tf.keras.utils.SidecarEvaluator`. Você também poderá visualizar as métricas de avaliação no TensorBoard, que não são mostradas neste guia.

Para ajudar a demonstrar isto, vamos começar definindo e treinando o modelo:


In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
  ])

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model = create_model()
model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'],
              steps_per_execution=10,
              run_eagerly=True)

log_dir = tempfile.mkdtemp()
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(log_dir, 'ckpt-{epoch}'),
    save_weights_only=True)

model.fit(x=x_train,
          y=y_train,
          epochs=1,
          callbacks=[model_checkpoint])

Em seguida, avalie o modelo usando `tf.keras.utils.SidecarEvaluator`. No treinamento real, é recomendável usar um job separado para conduzir a avaliação e liberar recursos do worker para o treinamento.

In [ ]:
data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
data = data.batch(64)

tf.keras.utils.SidecarEvaluator(
    model=model,
    data=data,
    checkpoint_dir=log_dir,
    max_evaluations=1
).start()

## Próximos passos

- Para saber mais sobre a avaliação secundária (sidecar), considere ler os documentos da API `tf.keras.utils.SidecarEvaluator`.
- Para considerar a alternância entre treinamento e avaliação no Keras, leia sobre [outros métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate).